In [1]:
import json 
import pandas as pd 

# Test model results 

In [2]:
test_model_results = []

with open("test_responses.jsonl",'r') as f : 
    for line in f: 
        if line.strip():
            data = json.loads(line)
            test_model_results.append(data)


In [3]:
df_test_model = pd.DataFrame(test_model_results)
df_test_model

,test model provider,test model version,Encountered Problems,test model response,sample_id
0,openai,gpt-3.5-turbo,False,Based on the information provided in the conte...,0
1,openai,gpt-4o,False,"Based on the context document, your risk facto...",0
2,openai,gpt-3.5-turbo,False,1. Wüsthof - 14 degrees\n2. MAC - 15 degrees\n...,1
3,openai,gpt-4o,False,"Based on the provided information, here is a l...",1


In [4]:
print(df_test_model['test model response'].iloc[0])

Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation. 

Cognitive engagement is important because it is thought to support the development of a "cognitive reserve," which may protect against brain cell damage caused by dementia. Engaging in activities that challenge and stimulate the brain throughout life may help reduce the risk of developing dementia.


In [5]:
print(df_test_model['test model response'].iloc[1])

Based on the context document, your risk factors for dementia include:

1. **Hearing Loss**: Having your ears blown out in the war may have led to hearing loss, which increases the risk of cognitive decline and dementia.
2. **Poor Diet**: Eating mostly garbage, which likely means an unhealthy diet high in saturated fat, sugar, and salt, can increase the risk of developing dementia.
3. **Depression**: Experiencing regular cases of "the sads" suggests depression, which is associated with a higher risk of developing dementia.
4. **Social Isolation**: If your hearing loss leads to social isolation, this can further increase the risk of dementia.

Regarding cognitive engagement, the document suggests that actively using your brain throughout life supports the development of a "cognitive reserve." This cognitive reserve may offer protection against brain cell damage caused by dementia, potentially reducing the risk or slowing the progression of the disease.


# Judges

In [6]:
judges_result = []

with open("judge_responses.jsonl",'r') as f : 
    for line in f:
        if line.strip():
            content = json.loads(line)
            judges_result.append(content)

In [7]:
df_judges = pd.DataFrame(judges_result)
df_judges

,judge model provider,judge model version,Encountered Problems,judge model response,judgeded_model,sample_id,verdict
0,anthropic,claude-3-5-sonnet-20240620,False,Sentence 1: Based on the information provided ...,openai:gpt-3.5-turbo,0,Accurate
1,google_vertexai,gemini-1.5-pro,False,"```json\n{""sentence"": ""Based on the informatio...",openai:gpt-3.5-turbo,0,Accurate
2,anthropic,claude-3-5-sonnet-20240620,False,"Sentence 1: Based on the context document, you...",openai:gpt-4o,0,Accurate
3,google_vertexai,gemini-1.5-pro,False,"```json\n{""sentence"": ""Based on the context do...",openai:gpt-4o,0,Accurate
4,anthropic,claude-3-5-sonnet-20240620,False,Sentence 1: Wüsthof - 14 degrees\nSentence 1 l...,openai:gpt-3.5-turbo,1,Inaccurate
5,google_vertexai,gemini-1.5-pro,False,"```json\n{""sentence"": ""1. Wüsthof - 14 degrees...",openai:gpt-3.5-turbo,1,Accurate
6,anthropic,claude-3-5-sonnet-20240620,False,"Sentence 1: Based on the provided information,...",openai:gpt-4o,1,Accurate
7,google_vertexai,gemini-1.5-pro,False,"```json\n{""sentence"": ""Based on the provided i...",openai:gpt-4o,1,Accurate


In [24]:
print(len(df_test_model))
print(len(df_judges))

4
8


# Explore some samples 

In [8]:
print(df_judges['judge model response'].iloc[0])

Sentence 1: Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation.
Sentence 1 label: Accurate

Sentence 2: Cognitive engagement is important because it is thought to support the development of a "cognitive reserve," which may protect against brain cell damage caused by dementia.
Sentence 2 label: Accurate

Sentence 3: Engaging in activities that challenge and stimulate the brain throughout life may help reduce the risk of developing dementia.
Sentence 3 label: Accurate

Final Answer: Accurate


In [9]:
print(df_judges['judge model response'].iloc[1])

```json
{"sentence": "Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation.", "label": "supported", "rationale": "All of the listed factors are mentioned as risk factors in the context.", "excerpt": "high blood pressure \n People who have consistent high blood pressure (hypertension) in mid-life (ages 45 to 65) are more likely to develop dementia compared to those with normal blood pressure.\n \n\n High blood pressure can increase the risk of developing dementia, particularly vascular dementia, because of its effect on the heart, the arteries, and blood circulation.\n \n\n Smoking \n The evidence is strong and consistent that smokers are at a higher risk of developing dementia vs. non-smokers or ex-smokers.\n \n\n It’s never too late to quit! S

# Overall Accuracy 

In [33]:
TEST_MODEL = 'openai:gpt-3.5-turbo' # feel free to checkout accuracy for other models 

In [34]:
df_judges[df_judges['judgeded_model']==TEST_MODEL].groupby('judge model version')['verdict'].value_counts().unstack(fill_value=0)

verdict,Accurate,Inaccurate
judge model version,,
claude-3-5-sonnet-20240620,1,1
gemini-1.5-pro,2,0


In [38]:
df_filtered = df_judges[df_judges['judgeded_model'] == TEST_MODEL]
counts = df_filtered.groupby('judge model version')['verdict'].value_counts()
percentages = counts / counts.groupby(level=0).sum() * 100
result = percentages.unstack(fill_value=0)[["Accurate"]]

result

verdict,Accurate
judge model version,
claude-3-5-sonnet-20240620,50.0
gemini-1.5-pro,100.0
